In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
data_dir = '/om2/user/rogerjin/data/SNARESeq/original/adult_brain_cortex'
cdna_path = f'{data_dir}/GSE126074_AdBrainCortex_SNAREseq_cDNA.counts.mtx'
cdna_genes_path = f'{data_dir}/GSE126074_AdBrainCortex_SNAREseq_cDNA.genes.tsv'
atac_path = f'{data_dir}/GSE126074_AdBrainCortex_SNAREseq_chromatin.counts.mtx'
cdna_barcodes_path = f'{data_dir}/GSE126074_AdBrainCortex_SNAREseq_cDNA.barcodes.tsv'

In [2]:
# import sys

# num_bytes = sys.getsizeof(rna)

# def sizeof_fmt(num, suffix="B"):
#     # https://stackoverflow.com/questions/1094841/get-human-readable-version-of-file-size
#     for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
#         if abs(num) < 1024.0:
#             return f"{num:3.1f}{unit}{suffix}"
#         num /= 1024.0
#     return f"{num:.1f}Yi{suffix}"

# sizeof_fmt(num_bytes)

In [3]:
import scanpy as sc

sc.set_figure_params(dpi=300)
sc._settings.ScanpyConfig.n_jobs = 4

In [4]:
import pandas as pd

rna = sc.read_mtx(cdna_path).T
cell_ids = pd.read_csv(cdna_barcodes_path, header=None)
cell_ids.index = cell_ids.index.map(str)
cell_ids.columns = ['cell_id']
cell_ids

,cell_id
0,09A_CAGCCCCGCCTT
1,09A_CGCCTACCATGA
2,09A_GATGCGCGGCTA
3,09A_GGTCCGAGTCCT
4,09A_TCTCCCGGCACC
...,...
10304,09L_TACTAGTTCAAG
10305,09L_ATGACGGGCCCC
10306,09L_GAAACACCTCAT
10307,09L_AACGGTTTATCC


In [5]:
labels_path = 'snareseq_ad_annot_labeled.csv'
labels = pd.read_csv(labels_path).drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
cell_types = labels[['cell_id', 'cell_type']]

In [ ]:
rna.obs = rna.obs.merge(cell_ids, left_index=True, right_index=True)

In [ ]:
import pandas as pd
rna_genes = pd.read_csv(cdna_genes_path, header=None)

In [ ]:
mito_genes = pd.read_csv('mitochondrial_genes.csv')
mito_genes

In [ ]:
# is_mito = rna_genes.apply(lambda gene: gene i
mito_gene_set = set(mito_genes.Symbol)
is_mito = rna_genes[0].apply(lambda gene: gene in mito_gene_set)

# compute % mitochondrial reads
rna.var_names = rna_genes[0]
rna.var['mt'] = rna.var_names.isin(mito_gene_set)

In [ ]:
rna.var['mt']

In [ ]:
sc.pp.calculate_qc_metrics(rna, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.highest_expr_genes(rna, n_top=20)

In [ ]:
sc.pp.filter_cells(rna, min_genes=200)
sc.pp.filter_genes(rna, min_cells=3)

In [ ]:
sc.pl.violin(rna, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(rna, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(rna, x='total_counts', y='n_genes_by_counts')

In [ ]:
rna = rna[rna.obs.n_genes_by_counts < 2500, :]
rna = rna[rna.obs.pct_counts_mt < 5, :]

In [ ]:
rna.obs['idx'] = rna.obs.index
merged = rna.obs.merge(cell_types, on='cell_id')
rna = rna[merged.idx, :]
rna.obs['cell_type'] = merged.cell_type.values

In [ ]:
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)
sc.pp.highly_variable_genes(rna, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(rna)

In [ ]:
sc.pp.regress_out(rna, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(rna, max_value=10)

In [ ]:
sc.tl.pca(rna, svd_solver='arpack')

In [ ]:
sc.pl.pca(rna, color='cell_type')

In [ ]:
sc.pl.pca_variance_ratio(rna, log=True)

In [ ]:
sc.pp.neighbors(rna, n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.leiden(rna)
sc.tl.paga(rna)
sc.pl.paga(rna, plot=False)
sc.tl.umap(rna, init_pos='paga')

In [ ]:
sc.pl.umap(rna, color='cell_type')

In [ ]:
rna.write('snareseq_rna.h5ad')

In [ ]:
import anndata as ad

rna = ad.read('snareseq_rna.h5ad')
rna

# ATAC

In [6]:
atac = sc.read_mtx(atac_path).T

In [7]:
import episcanpy.api as epi
from ganoli_plot import plot_umap

atac.obs = atac.obs.merge(cell_ids, left_index=True, right_index=True)

In [8]:
atac.obs['idx'] = rna.obs.index
merged = atac.obs.merge(cell_types, on='cell_id')
atac = atac[merged.idx, :]
atac.obs['cell_type'] = merged.cell_type.values

Trying to set attribute `.obs` of view, copying.


In [9]:
plot_umap(atac, label_name='cell_type') 

         Falling back to preprocessing with `sc.pp.pca` and default params.


... storing 'cell_type' as categorical


ValueError: Given components: '{}' are not valid. Please check. A valid example is `components='2,3'`

In [12]:
pc_range = range(1,41)
','.join([str(i) for i in pc_range])

'1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40'